# Tutorial: An introduction to property graph transformations

In this tutorial, you will learn the key concepts related to our property graph transformations:
* How to specify transformations of property graph with this framework (i.e., Understanding how we make use of Skolem functions to construct new property graphs.)
* How the rules in a single transformation interact with each other (i.e., How content of output elements can be jointly specified in several rules.)
* Understand the notion of *conflicts*, and how to deal with them.
* How this framework integrates with openCypher (i.e., How the rules are compiled into openCypher scripts and in which context they are executed.)
* What property graph transformations are capable of doing (i.e., The kind of constructs that can be expressed.)

## Part 1: Preliminaries


By default this notebook is configured to connect to a local Neo4j instance running inside a Docker container. This [notebook](./Tutorial_Connecting_Neo4j_Docker.ipynb) will guide you trought the process of setting up a local Docker container and connecting to it.

In [3]:
from dtgraph import Neo4jGraph, Rule, Transformation
hostname = "localhost"
password = ""
uri = f"bolt://{hostname}:7687"
graph = Neo4jGraph(uri, database="neo4j", username="", password=password)

For this tutorial, we will use a stripped version of the Movies dataset from Neo4j.

In [4]:
from dtgraph.scenarios.movies import Movies
Movies.load(graph)

Flushed database: Deleted 125 nodes, deleted 584 relationships, completed after 1425 ms.
Load scenario: Added 171 labels, created 171 nodes, set 564 properties, created 253 relationships, completed after 3456 ms.
